# 埼玉県越谷市報道発表資料

埼玉県越谷市が発表した新型コロナの患者情報のスクレイピングを行います。県内他の市町村から発表された情報は含まれていません。

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
import bs4
import re
import csv
import datetime
import requests
from collections import defaultdict

In [3]:
#export
urls = ['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/index.html']

In [4]:
#export
def trim_div(s):
    return re.sub('<div>|<div .+?>|</div>', '', s)

In [5]:
#export
def get_src(url):
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    s = trim_div(response.text)
    return bs4.BeautifulSoup(s, 'html.parser')

In [6]:
#export
def get_links(urls):
    domain = 'https://www.city.koshigaya.saitama.jp'
    filter = '新型コロナウイルス.+患者.+発生'
    links = []
    for url in urls:
        src = get_src(url)
        links.extend(src.find_all(lambda tag: tag.name == 'a' and re.search(filter, tag.get_text()) != None))
    return [[domain + tag['href'], tag.get_text()] for tag in links]

In [7]:
links = get_links(urls)
links

[['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei8.html',
  '越谷市内における新型コロナウイルス感染症患者の発生について(4月7日発表)'],
 ['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei7.html',
  '越谷市内における新型コロナウイルス感染症患者の発生について(4月5日発表)'],
 ['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei6.html',
  '越谷市内における新型コロナウイルス感染症患者の発生について(4月4日発表)'],
 ['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei5.html',
  '越谷市内における新型コロナウイルス感染症患者の発生について(4月2日発表)'],
 ['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei4.html',
  '越谷市内における新型コロナウイルス感染症患者の発生について(3月31日発表)'],
 ['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei3.html',
  '越谷市内における新型コロナウイルス感染症患者の発生について(3月20日発表)'],
 ['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei2.html',
  '越谷市

In [8]:
#export
def parse(src, url):
    
    def get_text(tag):
        return tag.get_text().strip()
    
    def get_contents(src):
        h = src.body.find_all(lambda tag: tag.name == 'h2')
        for h in src.body.find_all(lambda tag: tag.name == 'h2'):
            t = h.get_text()
            if '〜' in t: continue # "nn例目〜nn例目"
            if '例目' in t: break
        return h.previous_sibling.next_siblings
    
    def parse_h2(e):
        no = re.sub("\\D", "", get_text(e))
        return no if len(no) > 0 else '0'

    def parse_table(e):
        table = []
        for tr in e.find_all('tr'):
            row = []
            for td in tr.find_all('td'):
                row.append(get_text(td))
            table.append(' '.join(row))
        return table

    def parse_ul(e):
        return [get_text(li) for li in e.find_all('li')]
    
    def parse_ol(e):
        item = {}
        for li in e.find_all('li'):
            split = re.split('[：：]', get_text(li))
            item[split[0]] = split[1]
        return item
    
    def parse_div(e):
        ul = []
        for c in e.descendants:
            if c.name == 'p':
                ul.append(get_text(c))
        return ul
    
    def dic2array(i):
        return [i['概要'],i['年代'],i['性別'],i['国籍'],i['職業'],i['通勤'],i['居住地'],i['症状、経過'],i['その他'],i['url']]

    patients = []
    item = defaultdict(lambda: '')
    for e in get_contents(src):
        if isinstance(e, bs4.element.Tag):
            if e.name == 'h2':
                if 'お問い合わせ' in e.get_text(): break
                if len(item) > 0:
                    item['url'] = url
                    patients.append(dic2array(item))
                    item = defaultdict(lambda: '')
                item['概要'] = parse_h2(e)
            elif e.name == 'h3':
                pass
            elif e.name == 'table':
                item['症状、経過'] = '\n'.join(parse_table(e)).strip()
            elif e.name == 'ul':
                item['その他'] = '\n'.join(parse_ul(e))
            elif e.name == 'ol':
                item.update(parse_ol(e))
            elif e.name == 'p':
                pass
            else:
                print(type(e), e.name)
    if len(item) > 0:
        item['url'] = url
        patients.append(dic2array(item))
    return patients

In [9]:
#export
def get_patient(url):
    src = get_src(url)
    return parse(src, url)

In [10]:
#export
def get_patients(urls):
    patients = []
    for link in get_links(urls):
        print(link)
        patients.extend(get_patient(link[0]))
    return patients

In [11]:
def get_sample(urls, i):
    links = get_links(urls)
    return get_src(links[i][0])
#get_sample(urls, 0)

In [12]:
#export
def create_fname(base):
    now = datetime.datetime.now()
    return base + '_' + now.strftime('%Y%m%dT%H%M') + ".csv"

In [13]:
#export
def write_csv(patients, fname):
    with open(fname, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(patients)

In [14]:
#export
def main():
    patients = sorted(get_patients(urls), key=lambda x: int(x[0]))
    fname = create_fname("data/11-city-koshigaya")
    write_csv(patients, fname)
    print("created:", fname)

In [15]:
# See: https://github.com/fastai/course-v3/blob/master/nbs/dl2/notebook2script.py
!python notebook2script.py 11-city-koshigaya_dev.ipynb

Converted 11-city-koshigaya_dev.ipynb to exp/nb_11-city-koshigaya.py


In [16]:
#export
if __name__ == '__main__':
    main()

['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei8.html', '越谷市内における新型コロナウイルス感染症患者の発生について(4月7日発表)']
['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei7.html', '越谷市内における新型コロナウイルス感染症患者の発生について(4月5日発表)']
['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei6.html', '越谷市内における新型コロナウイルス感染症患者の発生について(4月4日発表)']
['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei5.html', '越谷市内における新型コロナウイルス感染症患者の発生について(4月2日発表)']
['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei4.html', '越谷市内における新型コロナウイルス感染症患者の発生について(3月31日発表)']
['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei3.html', '越谷市内における新型コロナウイルス感染症患者の発生について(3月20日発表)']
['https://www.city.koshigaya.saitama.jp/kurashi_shisei/fukushi/hokenjo/kansensho/kansen_hassei2.html', '越谷市内における新型コロナウイルス感染症患者の発生について(